In [1]:
# -*- coding:utf-8 -*-
import tensorflow as tf
from tensorflow import keras
import numpy as np
import random
import cv2
import os

orignal =[] #Blank array. But later on add normal and abnormal to it. original has all the labels we got during training.


In [2]:
#Read all test videos
test=[] #It contains all labels (0 and 1). Length is 85 
test_list2=[]
f= 'sample_images_RA'
Test_videos =[]  #This array contains path of all videos.Length is 85
test_list = open('Labels/test.txt','r') # path and labels info 
for line in test_list:
    name = line.split(" ")[0]  
    test_list2.append(line)
    x=line.split(" ")[-1]
    test.append(x[0])
    Test_videos.append(f+"/"+name) #Full path of all videos
    

In [3]:
for i in test: 
    if i=='0':
        orignal.append('normal')
    else:
        orignal.append('abnormal')

In [4]:
#To load pre-trained model, trained on 5 classes 
#model = tf.keras.models.load_model('Trained models/model_c3d.h5',compile=False)
#model.summary()


#To load pre-trained weeights,
model = tf.keras.models.load_model('Trained Models/weights_model_C3D.hdf5',compile=False)


In [5]:
#Five class labels
labels1 = {0: 'fighting', 1: 'vandalism',2: '0normal',3: 'shooting',4: 'hockeyfight'}
prediction = [] #Labels you get after testing

In [6]:
def add_motion_blur(frame, kernel_size=5, orientation='horizontal'):
    kernel = np.zeros((kernel_size, kernel_size))
    
    if orientation == 'horizontal':
        # Horizontal motion blur (default)
        kernel[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)
    elif orientation == 'vertical':
        # Vertical motion blur
        kernel[:, int((kernel_size - 1) / 2)] = np.ones(kernel_size)
    elif orientation == 'diagonal':
        # 45-degree diagonal motion blur
        for i in range(kernel_size):
            kernel[i, i] = 1
        kernel = kernel / kernel_size
    elif orientation == 'diagonal_135':
        # 135-degree diagonal motion blur
        for i in range(kernel_size):
            kernel[i, -(i + 1)] = 1
        kernel = kernel / kernel_size
    elif orientation == 'cross':
        # Cross motion blur (horizontal and vertical)
        kernel[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)
        kernel[:, int((kernel_size - 1) / 2)] = np.ones(kernel_size)
        kernel = kernel / kernel_size
    else:
        raise ValueError("Invalid orientation. Use 'horizontal', 'vertical', 'diagonal', 'diagonal_135', or 'cross'.")

    blurred_frame = cv2.filter2D(frame, -1, kernel)
    return blurred_frame

In [7]:
def play_video(frames, frame_rate=5):
    """
    Play a sequence of frames as a video.

    Parameters:
    - frames: List of frames (numpy arrays representing images).
    - frame_rate: Frames per second (default is 25).
    """
    if not frames:
        print("No frames to play.")
        return

    # Get the height and width from the first frame
    height, width, _ = frames[0].shape
    size = (width, height)

    # Create a VideoWriter object to save the video
    video_writer = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'DIVX'), frame_rate, size)

    # Display each frame and write it to the video file
    for frame in frames:
        cv2.imshow('Video Player', frame)
        video_writer.write(frame)

        # Break the loop if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoWriter and close the OpenCV window
    video_writer.release()
    cv2.destroyAllWindows()

In [7]:
all_videos =[]

#Code for Confusion Matrix
for video in Test_videos:
    vid = [] #reads frame of videos
    #nor=0  #no value of nor. initialize nor with 0
    abnor=0   #no value of abnor. initialize abnor 0
    path=video
    images = os.listdir(path) #Images name actually store name of all frames
    for img in images: #read the length and width of all images.
        img_path=path +"/"+img #path +img name print. path of 1 frame
        img2 = cv2.imread(img_path) #img2 has a 2D array of integer form containing length and width of videos
        img2 = add_motion_blur(img2, kernel_size=30, orientation='cross')
        img2= cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        vid.append(cv2.resize(img2, (1024,1024))) 
    all_videos.append(vid)

In [9]:
play_video(all_videos[0])